# Scrape Impacespace Impace Investment data
 - 3 hours main time focus on parsing the data with html parser, problem arise when use package parser pd.read_html and beautifulsoup
 - manual parser works with unknown reason

In [183]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from html_table_extractor.extractor import Extractor
import numpy as np

## read_html from pandas

In [2]:
# forbidden website
pd.read_html('https://impactspace.com/investments')

HTTPError: HTTP Error 403: Forbidden

In [90]:
url = 'https://impactspace.com/investments'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

df_list = pd.read_html(r.text
                        , converters={'Size':object,'Date':object} #prevent pandas to convert this column to numeric
                        # , encoding='utf-8'
                        )

TypeError: object() takes no arguments

In [87]:
df_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4936 entries, 0 to 4935
Data columns (total 7 columns):
Company                   4763 non-null object
Financial Organization    4586 non-null object
Date                      2391 non-null object
Stage                     1823 non-null object
Size                      0 non-null object
Vehicle                   1152 non-null object
Deal Type                 1752 non-null object
dtypes: object(7)
memory usage: 270.1+ KB


In [88]:
df_list[0].head()

,Company,Financial Organization,Date,Stage,Size,Vehicle,Deal Type
0,Dvara KGFS,LeapFrog Investments,NaN,Series C +,NaN,Equity,Joint
1,Dvara KGFS,Dvara Trust,NaN,Series C +,NaN,Equity,Joint
2,Dvara KGFS,ACCION,NaN,Series C +,NaN,Equity,Joint
3,Dvara KGFS,Stakeboat Capital,NaN,Series C +,NaN,Equity,Joint
4,Mfine,SBI investment,NaN,Series B,NaN,Equity,NaN


In [91]:
df_list[0].Date.unique()

array([nan, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010,
       2009, 2008, 2007, 2006, 2005, 2004, 2003, 1982, 0], dtype=object)

In [22]:
# normal html from request
r.text[10000:20000]

'eadersearch">\r\n\t\t<span style="display:none; float:top; top: 228px !important; width: 265px; left: 979px; max-height:450px; overflow-y:scroll;" id="autocompleteC"></span></div>\r\n    </div><!-- end subheader container -->\r\n</section><!-- end subheader section -->\r\n\r\n \r\n<script type="text/javascript" src="https://impactspace.com/js/dtjquery.js"></script>\r\n<script type="text/javascript" src="https://impactspace.com/js/jquery.dataTables.js"></script>\r\n<script type="text/javascript" src="https://impactspace.com/js/datatable.js"></script>\r\n\r\n         <div id="wrapper" class="container" style="margin-top: 35px;-moz-user-select: none; -webkit-user-select: none; -ms-user-select:none; user-select:none;-o-user-select:none;" unselectable="on" onselectstart="return false;" onmousedown="return false;" oncopy="return false" oncut="return false" onpaste="return false">\r\n\t<p>Below is a list of all current deals. If you\'d like to add a new deal, please visit the <a href=\'/fina

In [75]:
print("Length of this page code(in html): {0} characters".format(len(r.text)))

Length of this page code(in html): 2852006 characters


In [15]:
df_list[0]['Financial Organization'].nunique()

978

## BeautifulSoup
- seems like the parser has some problem with parsing part of the data.

In [66]:
r.text

>Education Development Center</a></td>\r\n\r\n\t\t    <td>\r\n                                 \r\n                       <a href="/financial-organization/the-annie-e-casey-foundation">The Annie E. Casey Foundation</a></td>\r\n\t\t    <td>0000</td>\r\n\t\t    <td style=\'word-break:break-all;\'></td>\r\n\t\t    <td>N/A                              \r\n                              </td>\r\n\t\t    <td align="left" ></td>\r\n\t\t    \t\t    <td align="left" >Joint</td>\r\n\t\t</tr>\r\n\t\t<tr>\r\n                  <td>\r\n                    \t\t    <a href="/company/varthana">Varthana</a></td>\r\n\r\n\t\t    <td>\r\n                                 \r\n                       <a href="/financial-organization/accion-venture-lab">Accion Venture Lab</a></td>\r\n\t\t    <td>0000</td>\r\n\t\t    <td style=\'word-break:break-all;\'></td>\r\n\t\t    <td>N/A                              \r\n                              </td>\r\n\t\t    <td align="left" ></td>\r\n\t\t    \t\t    <td align="left

In [67]:
# get all the web elements
all_web_element=BeautifulSoup(markup=r.text
                            , features='html.parser'
                            )

In [72]:
all_web_element

KeyError: 0

In [50]:
x=all_web_element.find(name='table'
                        , class_='dataTable'
                        )
x

In [52]:
x=all_web_element.find(name='tbody'
                        # , class_='dataTable'
                        )
x

In [55]:
all_web_element.findall(name='tbody')

TypeError: 'NoneType' object is not callable

In [58]:
type(all_web_element)

bs4.BeautifulSoup

In [ ]:
x.find()

## Manual Extract with html_table_extractor.extractor import Extractor
- https://stackoverflow.com/questions/51349215/pandas-read-html-clean-up-before-or-after-read

In [104]:
beg=r.text.rfind('<table') # beginning
end=r.text.rfind('/table>') # ending

In [107]:
beg

11220

In [133]:
# Clean column name
[i.lower().replace(' ','_') for i in df_list[0].columns]

['company',
 'financial_organization',
 'date',
 'stage',
 'size',
 'vehicle',
 'deal_type']

In [134]:
# Manual parse
ex=Extractor(r.text[beg:end])
ex.parse()
line_list=ex.return_list()

df=pd.DataFrame(line_list,columns=[i.lower().replace(' ','_') for i in df_list[0].columns])
df

,company,financial_organization,date,stage,size,vehicle,deal_type
0,\nDvara KGFS,\nLeapFrog Investments,201905\r\n\t\t \t\t\t05/19,Series C +,99914000000\r\n ...,Equity,Joint
1,\nDvara KGFS,\nDvara Trust,201905\r\n\t\t \t\t\t05/19,Series C +,99914000000\r\n ...,Equity,Joint
2,\nDvara KGFS,\nACCION,201905\r\n\t\t \t\t\t05/19,Series C +,99914000000\r\n ...,Equity,Joint
3,\nDvara KGFS,\nStakeboat Capital,201905\r\n\t\t \t\t\t05/19,Series C +,99914000000\r\n ...,Equity,Joint
4,\nMfine,\nSBI investment,201905\r\n\t\t \t\t\t05/19,Series B,99917200000\r\n ...,Equity,
...,...,...,...,...,...,...,...
4931,\nMagic Bus,\nDasra Social Impact,0000,,N/A \r\n ...,,
4932,\nEco Fuels Kenya,\nADAP Capital LLC,0000,,N/A \r\n ...,,
4933,\nRuma,\nPatamar Capital,0000,,N/A \r\n ...,,Joint
4934,\nDemyst Data,\nAccion Venture Lab,0000,,N/A \r\n ...,,Joint


In [148]:
df['date'].apply(lambda x: x[:7]).unique()

array([' 201905', ' 201904', ' 201903', ' 201902', ' 201901', '2019',
       ' 201810', ' 201809', ' 201808', ' 201807', ' 201806', ' 201805',
       ' 201804', ' 201803', ' 201802', ' 201801', '2018', ' 201709',
       ' 201708', ' 201707', ' 201706', ' 201705', ' 201704', ' 201703',
       ' 201702', ' 201701', '2017', ' 201612', ' 201611', ' 201610',
       ' 201609', ' 201608', ' 201607', ' 201606', ' 201605', ' 201604',
       ' 201603', ' 201602', ' 201601', '2016', ' 201512', ' 201511',
       ' 201510', ' 201509', ' 201508', ' 201507', ' 201506', ' 201505',
       ' 201504', ' 201503', ' 201502', ' 201501', '2015', ' 201412',
       ' 201411', ' 201410', ' 201409', ' 201408', ' 201407', ' 201406',
       ' 201405', ' 201404', ' 201403', ' 201402', ' 201401', '2014',
       ' 201312', ' 201311', ' 201310', ' 201309', ' 201308', ' 201307',
       ' 201306', ' 201305', ' 201304', ' 201303', ' 201302', ' 201301',
       '2013', ' 201212', ' 201211', ' 201210', ' 201209', ' 201208',

In [325]:
# Clean df
def since3(text):
    return text[1:]

def clean(text):
    if(len(text)==0):
        return np.NaN
    else:
        if(text[0][-1]=='M'):
            location_currency1=text[0].find('$')
            location_currency2=text[0].find('€')
            location_currency3=text[0].find('£')
            location_currency=max(location_currency1, location_currency2,location_currency3)
            return (text[0][location_currency],float(text[0][location_currency+1:-1])*10**6) # (currency, value)
        elif(text[0][-1]=='K'):
            location_currency1=text[0].find('$')
            location_currency2=text[0].find('€')
            location_currency3=text[0].find('£')
            location_currency=max(location_currency1, location_currency2,location_currency3)
            return (text[0][location_currency],float(text[0][location_currency+1:-1])*10**3) # (currency, value)

def clean_date(string_date, method):
    # QA
    methods=['year','month']
    if(method not in methods):
        raise Exception(f'{method} is not specified, please input {methods}.')

    # Clean Missing data with NULL
    if(int(string_date)<1):
        return np.NaN

    if(method=='year'):
        return string_date[:4]
    else:
        return string_date[4:]

df_clean=pd.DataFrame()
# df['company'].apply(lambda x: since3(x)) # test clean
df_clean[['company','financial_organization']]=df[['company','financial_organization']].applymap(lambda x: since3(x))
df_clean['raw_date']=df['date'].apply(lambda x: x[1:7])
df_clean['year']=df_clean['raw_date'].apply(lambda x: clean_date(x,'year'))
df_clean['month_id']=df_clean['raw_date'].apply(lambda x: clean_date(x,'month'))
df_clean['stage']=df['stage'].apply(lambda x: 'Unspecified' if(len(x)==0) else x)
df_clean['fund_size_tuple']=df['size'].apply(lambda x: x[20:80].split()).apply(lambda x: clean(x))
df_clean[['currency_sign','fund_amount']]=pd.DataFrame(df_clean.fund_size_tuple.tolist())
df_clean['financial_organization']=df_clean.apply(lambda x: 'Missing org name' if(x.fund_amount>0 and len(x.financial_organization)==0) else x.financial_organization,axis=1)

In [297]:
df.loc[1000:1002,'size'].apply(lambda x: x[20:80].split()[0].find('$'))

1000    0
1001    0
1002    1
Name: size, dtype: int64

In [305]:
df_clean

,company,financial_organization,raw_date,year,month_id,stage,fund_size_tuple,currency_sign,fund_amount
0,Dvara KGFS,LeapFrog Investments,201905,2019,05,Series C +,"($, 14000000.0)",$,14000000.0
1,Dvara KGFS,Dvara Trust,201905,2019,05,Series C +,"($, 14000000.0)",$,14000000.0
2,Dvara KGFS,ACCION,201905,2019,05,Series C +,"($, 14000000.0)",$,14000000.0
3,Dvara KGFS,Stakeboat Capital,201905,2019,05,Series C +,"($, 14000000.0)",$,14000000.0
4,Mfine,SBI investment,201905,2019,05,Series B,"($, 17200000.0)",$,17200000.0
...,...,...,...,...,...,...,...,...,...
4931,Magic Bus,Dasra Social Impact,000,NaN,NaN,Unspecified,NaN,NaN,NaN
4932,Eco Fuels Kenya,ADAP Capital LLC,000,NaN,NaN,Unspecified,NaN,NaN,NaN
4933,Ruma,Patamar Capital,000,NaN,NaN,Unspecified,NaN,NaN,NaN
4934,Demyst Data,Accion Venture Lab,000,NaN,NaN,Unspecified,NaN,NaN,NaN


In [ ]:
df

In [326]:
df_final=df_clean[['company','financial_organization','year','month_id','stage','currency_sign','fund_amount']]
df_final

,company,financial_organization,year,month_id,stage,currency_sign,fund_amount
0,Dvara KGFS,LeapFrog Investments,2019,05,Series C +,$,14000000.0
1,Dvara KGFS,Dvara Trust,2019,05,Series C +,$,14000000.0
2,Dvara KGFS,ACCION,2019,05,Series C +,$,14000000.0
3,Dvara KGFS,Stakeboat Capital,2019,05,Series C +,$,14000000.0
4,Mfine,SBI investment,2019,05,Series B,$,17200000.0
...,...,...,...,...,...,...,...
4931,Magic Bus,Dasra Social Impact,NaN,NaN,Unspecified,NaN,NaN
4932,Eco Fuels Kenya,ADAP Capital LLC,NaN,NaN,Unspecified,NaN,NaN
4933,Ruma,Patamar Capital,NaN,NaN,Unspecified,NaN,NaN
4934,Demyst Data,Accion Venture Lab,NaN,NaN,Unspecified,NaN,NaN


## Analysis

In [308]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4936 entries, 0 to 4935
Data columns (total 7 columns):
company                   4936 non-null object
financial_organization    4936 non-null object
year                      3307 non-null object
month_id                  3307 non-null object
stage                     4936 non-null object
currency_sign             2019 non-null object
fund_amount               2019 non-null float64
dtypes: float64(1), object(6)
memory usage: 270.1+ KB


In [313]:
df_final.currency_sign.value_counts()

$    2004
£      13
€       2
Name: currency_sign, dtype: int64

In [312]:
df_final.stage.value_counts()

Unspecified    3113
Seed           1110
Series A        309
Series B        226
Series C +      177
Series C          1
Name: stage, dtype: int64

In [328]:
print("TOP 10 financial organization that give fund")
df_final.groupby('financial_organization').fund_amount.sum().reset_index().sort_values('fund_amount',ascending=False).head(10)

TOP 10 financial organization that give fund


,financial_organization,fund_amount
572,Missing org name,2.989169e+09
645,Overseas Private Investment Corporation,2.092811e+09
122,Bill and Melinda Gates Foundation,8.303300e+08
633,Obvious Ventures,3.952000e+08
64,Andreessen Horowitz,3.740900e+08
890,UBS,3.250000e+08
208,Community Investment Management,3.100000e+08
498,Kleiner Perkins,3.072100e+08
815,Tao Capital Partners,2.942100e+08
636,Omidyar Network,2.838250e+08


In [344]:
(
    df_final
    .groupby(['stage'])
    .agg({'fund_amount':sum, 'company':['count','nunique']})
    .reset_index()
    .sort_values(('fund_amount','sum'),ascending=False)
)

stage   fund_amount company        
                         sum   count nunique
5  Unspecified  1.119602e+10    3113    1853
4   Series C +  6.168943e+09     177      50
2     Series B  4.213893e+09     226      86
1     Series A  2.683902e+09     309     139
0         Seed  6.540787e+08    1110     712
3     Series C  0.000000e+00       1       1

In [331]:
df_final[df_final.financial_organization=='']

,company,financial_organization,year,month_id,stage,currency_sign,fund_amount
561,Bright Reality,,2017,07,Unspecified,NaN,NaN
587,Solve Smart Cities,,2017,07,Seed,NaN,NaN
811,,,2016,10,Unspecified,NaN,NaN
824,SpringU,,2016,08,Seed,NaN,NaN
827,Workit Health,,2016,08,Seed,NaN,NaN
...,...,...,...,...,...,...,...
4630,BlocPower,,NaN,NaN,Unspecified,NaN,NaN
4819,BushProof,,NaN,NaN,Unspecified,NaN,NaN
4894,SustainTech India,,NaN,NaN,Unspecified,NaN,NaN
4908,,,NaN,NaN,Unspecified,None,NaN
